# 스트림 처리의 기초
- 스파크는 스트림 처리를 위한 다음 두 가지 API를 도입함
  - DStream(Discretized Stream)
  - 구조적 스트리밍
  
- DStream
  - 자바나 파이썬 객체에 대한 <strong>저수준 연산</strong>만 지원
  - 따라서 고수준 최적화 기법을 활용하는데 한계가 있음
  
- 구조적 스트리밍
  - <strong>DataFrame, Dataset</strong> 기반
  - RDD와 DataFrame의 관계처럼 DStream의 주요 기능에 대한 상위 기능을 제공함

## 스트림 처리란
- 스트림 처리(stream processing)는 <strong>신규 데이터를 끊임없이 처리</strong>해 결과를 만들어내는 행위
  - <strong>입력 데이터는 무한</strong>
  - 시작과 끝을 사전에 정의하지 않음
  - 입력데이터의 예시로, 신용카드 전표 정보, 웹사이트 클릭, IoT장비의 센서 데이터 등이 있음
  - 반면, 배치처리는 입력데이터 크기가 미리 정해져있고, 결과를 한 번만 만들어냄

- 스트리밍 애플리케이션은 <strong>이벤트 스트림이 도착한 후 </strong>다양한 쿼리 연산을 수행함
  - ex) 이벤트 타입별 실행 카운트 추적, 시간별로 이벤트의 타입별 실행 카운트 집계 등

- 스트리밍 애플리케이션은 실행하면서 <strong>다양한 버전의 결과를 출력</strong>하거나 키-값 저장소 같은 <strong>외부 'sink' 시스템에 최신 데이터를 저장</strong>할 수도 있음

### 배치 처리와 스트림 처리를 함께 사용하는 경우
- <strong>스트림 입력 데이터를 배치 작업에서 주기적으로 만들어내는 데이터셋과 조인</strong>하는 경우
- <strong>스트리밍 작업의 출력이 배치 작업용 쿼리에 필요</strong>한 경우
- 주의: 모든 비즈니스 로직은 스트리밍과 배치 연산에서 <strong>일관성</strong> 있게 동작해야함
  - 구조적 스트리밍은 배치 애플리케이션 뿐만 아니라 나머지 컴포넌트와 <strong>쉽게 연동</strong> 가능
  - 연속형 애플리케이션: 스트리밍, 배치, 대화형 작업으로 구성된 통합 애플리케이션

### 스트림 처리 사례

#### 통보와 알림
- 실시간으로 특정 이벤트나 이벤트의 패턴을 탐지했을 때 알림

#### 실시간 리포트
- 실시간 대시보드를 만들 때
  - ex) 플랫폼 전체 사용량, 시스템 부하, 실행 시간, 신규 기능 사용량 등을 관찰하기 위한 대시보드

#### 증분형 ETL
- 데이터 웨어하우스에서 정보를 얻는 시간을 줄이기 위해

#### 실시간 제공용 데이터 갱신
- 다른 애플리케이션의 서비스용 데이터를 만들기 위해
  - ex) 웹 분석 서비스: 각 페이지의 방문자 수를 연속적으로 추적하여 최신 방문자 수를 갱신할 때 스트리밍 시스템 사용

#### 실시간 의사결정
- 실시간으로 신규 입력을 분석하고 자동으로 비즈니스 로직에 따라 처리하기 위해
  - ex) 신용카드 고객의 최근 이력을 기준으로 부정행위 여부 자동 판단

#### 온라인 머신러닝
- 실시간 데이터와 이력 데이터를 조합하여 모델 학습

### 스트림 처리의 장점
- <strong>대기 시간이 짧음</strong>
  - 빨리 응답해야하는 애플리케이션이면 <strong>상태를 메모리에 저장</strong>하는 스트리밍 시스템이 필요
  
- <strong>결과 수정 용이</strong>
  - ex) 24시간의 웹 트래픽 통계를 계산하고싶음
    - 배치 처리: 실행할 때마다 전체 데이터를 <strong>읽고 매번 24시간치 데이터를 계산함</strong>
    - 스트림 처리: 이전 연산의 상태를 기억하고 <strong>신규 데이터만 계산함</strong>

### 스트림 처리의 과제
- <strong>순서가 뒤섞인 데이터</strong> 처리하기(순서가 중요한 경우)
- <strong>대규모 상태 정보 유지</strong>하기
- <strong>높은 데이터 처리량</strong> 보장하기
- 장애 상황에서도 <strong>정확히 한 번 처리</strong>하기
- <strong>부하 불균형과 뒤처진 서버</strong> 다루기
- 이벤트에 <strong>빠르게 응답</strong>하기
- 다른 저장소 시스템의 <strong>외부 데이터와 조인</strong>하기
- 신규 이벤트 도착 시 출력 싱크의 <strong>갱신 방법 결정</strong>하기
- 출력 시스템에 데이터 저장 시 <strong>트랜잭션 보장</strong>하기
- <strong>런타임에 비즈니스 로직 변경</strong>하기